<a href="https://colab.research.google.com/github/mohit-sentieo/sentieo-public-api-examples/blob/master/Sentieo_Search_Public_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Search API Public Demo 
In this Notebook, we will show you how to use the search api. First we cover the basic features. Then we provide specific examples for edgar filings and notes to demonstrate more advanced features. Then we demonstrate some common errors and how to handle them.

For each block of code read the text above the block and then run the code.

In [0]:
!pip install ipywidgets 
!jupyter nbextension enable --py widgetsnbextension
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


### API CODE

This is the code which makes all the requests to the Sentieo API. It is structured like a class with functions for different functions. This does not have all the apis but only those required for this use case. Feel free to copy this class and the model directly into your code. 

Please fill in your own Api Key and Email at the top and run the block.

In [0]:
from typing import NamedTuple, List, Mapping
from datetime import date
import requests, json
from enum import Enum

APIKEY = '3syZzc0Epp4r52omw9M6U2owGBh5byxZ2EzTs67w'
BASE_URL = 'https://api.sentieo.com/v1'
EMAIL = 'mohit.kumar@sentieo.com'
default_headers = {
                  'x-api-key': APIKEY
                }

class DocTypeEnum(Enum):
  EDGARFILINGS = 'ef'
  PRESENTATIONS = 'ppt'
  PRESS_RELEASES = 'ni'
  NEWS = 'nw'
  GLOBAL_FILINGS = 'gbf'
  BROKER_RESEARCH = 'rr'
  TRANSCRIPTS = 'tt'
  NOTES = 'note'

  @classmethod
  def get_type_names(cls):
    return [ m.name for m in cls.__members__.values() ]
  
  def get_subtypes(self):
    try:
      with open('./subtypes/{0}.json'.format(self.value),'r') as f:
        subtypes = json.load(f)
    except:
      return []
    return [ st for v in subtypes.values() for st in v]


class SearchParams(NamedTuple):
  size:int = 20
  start:int = 0
  tickers:List[str] = []
  use_synonyms:bool = False
  query:str = ''
  date_from:date = None
  date_to:date = None
  doc_type:List[DocTypeEnum] = []
  doc_subtypes:Mapping[str,List[str]] = {}


class SentieoAPI:
  def __init__(self, debug=True):
    self.debug = debug

  def throttled_request(self, *args, **kwargs):
    response = requests.request(*args, **kwargs)
    if response.status_code == 429:
      time.sleep(0.2)
      response = requests.request(*args, **kwargs)
    return response

  def _make_post_api_call(self, url, payload=None, headers=None):
    payload = {} if payload is None else payload
    headers = { 'Content-Type': 'application/json' } if headers == None else headers
    headers.update(default_headers)
    response = self.throttled_request("POST", BASE_URL + url, headers=headers, data = json.dumps(payload))
    if self.debug:
      print(url, response.status_code)
    return response.status_code, response.json()
  
  def _make_get_api_call(self, url, params=None, headers=None):
    params = {} if params is None else params
    headers = { } if headers == None else headers
    headers.update(default_headers)
    response = self.throttled_request("GET", BASE_URL + url, headers=headers, params = params)
    if self.debug:
      print(url, response.status_code)
    return response.status_code, response.text

  def fetch_docs_from_search(self, filters:SearchParams=None):
    #date_range_from.value.strftime("%d-%b-%Y")
    filters = filters._asdict()
    if filters['date_from'] is not None:
      filters['date_from'] = filters['date_from'].strftime("%d-%b-%Y")
    if filters['date_to'] is not None:
      filters['date_to'] = filters['date_to'].strftime("%d-%b-%Y")
    filters['doc_type'] = [ m.value for m in filters['doc_type'] ]
    if 'all' in filters['doc_type']:
      del filters['doc_type']
    # print("Sending Payload", filters)
    status, result = self._make_post_api_call('/documents/search', filters)
    return result
  
  def fetch_doc_content(self, doc_id):
    status, result = self._make_get_api_call('/documents/get', { 'id': doc_id })
    return result


sAPI = SentieoAPI()

### UI CODE
This code is only for demo purposes. It handles the UI which is rendered below.

In a typical scenario, you will have your own UI built using Javascript which will send requests to your server. You server will then communicate with sentieo API. But since we cannot have that in a notebook, we have created a very simple UI in python itself.

Run this code and UI will render. You can then interact with the UI. To refresh the UI, run this code again.

In [0]:
tickers_ui = widgets.Text(
    value='msft, aapl',
    placeholder='Tickers seperated by commas',
    description='Tickers',
    disabled=False
)
start_ui = widgets.BoundedIntText(
    value=10,
    min=0,
    max=10,
    step=1,
    description='Skip docs',
    disabled=False
)
size_ui = widgets.BoundedIntText(
    value=10,
    min=0,
    max=2000,
    step=1,
    description='Total Docs',
    disabled=False
)
synonyms_ui = widgets.Checkbox(
    value=False,
    description='USE SYNONYMS',
    disabled=False,
    indent=False
)
sort_ui = widgets.Dropdown(
    options=['filing_date:desc' ,'filing_date:asc', 'score:desc', 'score:asc'],
    value='filing_date:desc',
    description='Order By',
    disabled=False,
)
query_ui = widgets.Text(
    value='',
    placeholder='in:title sales',
    description='Query',
    disabled=False
)
doc_type_ui = widgets.SelectMultiple(
    options=DocTypeEnum.get_type_names(),
    #rows=10,
    description='document types',
    disabled=False
)
doc_subtype_ui = widgets.SelectMultiple(
    options=[],
    description='Subtypes',
    disabled=True
)
date_to_ui = widgets.DatePicker(
    description='End Date',
    disabled=False
)
date_from_ui = widgets.DatePicker(
    description='Start Date',
    disabled=False
)

submit_button = widgets.Button(
    description='Fetch Documents',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
)

doc_id_ui = widgets.Text(
    value='',
    placeholder='4ejak2njfi',
    continuous_update=False,
    description='Enter a Doc Id to load',
    disabled=False,
    layout={"width": "300px", "margin": "50px"}
)


output_html = widgets.Output(layout={'border': '1px solid black'})

document_html = widgets.Output(layout={'border': '1px solid black'})
document_html.layout.height = '500px'

def doc_id_handler(change):
  if change['name'] == 'value' and len(change['new']) > 23 :
    document_html.clear_output(wait=True)
    doc_html = sAPI.fetch_doc_content(change['new'])
    doc_html = '<div style="background:white;font-color:black;">' + doc_html + '</div>'
    with document_html:
      display(widgets.HTML(value=doc_html.replace("class=\"t ", "class=\" ")))

def doc_subtype_handler(change):
  if change['name'] == 'value' and len(change['new']) > 0:
    doc_subtype_ui.options = DocTypeEnum[change['new'][0]].get_subtypes()
    doc_subtype_ui.description = change['new'][0]
    doc_subtype_ui.disabled = False
    

def build_form_ui():
  date_layout = widgets.HBox([date_from_ui, date_to_ui])
  text_layout = widgets.HBox([query_ui, tickers_ui])
  synonyms_layout = widgets.HBox([synonyms_ui, sort_ui])
  size_layout = widgets.HBox([start_ui, size_ui])
  doc_type_layout = widgets.HBox([doc_type_ui,doc_subtype_ui])
  doc_type_ui.observe(doc_subtype_handler)
  display(widgets.VBox([text_layout, date_layout, synonyms_layout, size_layout, doc_type_layout, submit_button]))

def doc_to_html(doc):
  return "<div style=\"border: 2px solid #888; padding: 5px; margin:5px;max-width: 300px\"><h4>{title}</h4><p>#{doc_id}</p><p> {ticker}, {country} </p> <p>{doc_subtype}, {doc_type}</p></div>".format(**doc)

def html_separator():
  return "<br><br>"

def enclose_doc_list(html):
  return '<div style="width:300px;height:600px;overflow:scroll;margin:100px;background:white;">' + html + '</div>'

def submit_callback(event):
  output_html.clear_output(wait=True)
  doc_type = [ DocTypeEnum[dt] for dt in doc_type_ui.value ] 
  searchparam = SearchParams(tickers=tickers_ui.value.strip().split(","), query=query_ui.value, start=start_ui.value, size=size_ui.value, date_from=date_from_ui.value, date_to=date_to_ui.value, doc_type=doc_type, use_synonyms=synonyms_ui.value)
  docs = sAPI.fetch_docs_from_search(searchparam)
  doc_list_html = html_separator().join([ doc_to_html(doc) for doc in docs['result']['docs'] ])
  final_html = enclose_doc_list(doc_list_html)
  html_widget = widgets.HTML(
      value=final_html
  )
  html_widget.layout.height = "400px"
  html_widget.layout.background = "white"
  with output_html:
    display(html_widget)
  if len(docs['result']['docs']) > 0:
    doc_id_ui.value = docs['result']['docs'][0]['doc_id']

submit_button.on_click(submit_callback)
build_form_ui()
display(doc_id_ui)
doc_id_ui.observe(doc_id_handler)
display(widgets.HBox([ output_html, document_html ]))

Text(value='', continuous_update=False, description='Enter a Doc Id to load', layout=Layout(margin='50px', wid…

/documents/search 200
/documents/get 200
